# Playlist Recommendation System

This notebook can be used to recommend songs when given a playlist of one or more songs. 

In [12]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
from scipy.sparse import csr_matrix
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
df = pd.read_json('pre_pivot.json')

In [14]:
df.shape

(515096, 13)

In [15]:
df.artist_and_track.nunique()

116157

In [16]:
df.artist_and_track.nunique()

116157

In [17]:
df.shape

(515096, 13)

In [18]:
df.added_by_id.nunique()

918

In [19]:
df.playlist_id.nunique()

6049

In [20]:
df.head()

,added_by_id,album_id,album_name,artist_ids,artist_names,date_added,playlist_id,track_id,track_name,user_id,rating,artists_join,artist_and_track
0,brendan.ta,4uIDigk79DeZEYV6Z5Yf4s,What Went Down,[6FQqZYVfTNQ1pCqfkwVFEa],[Foals],2016-03-26T00:48:16Z,3pTPXB3vT93AOTSVozP54o,53L6A3I9vf7rgEZnMzx54E,Mountain At My Gates,brendan.ta,1,Foals,Foals|||||Mountain At My Gates
1,brendan.ta,4sFhah3DYcJlYeT47q3rhM,In The Silence (Deluxe Version),[7xUZ4069zcyBM4Bn10NQ1c],[Ásgeir],2016-03-26T00:49:53Z,3pTPXB3vT93AOTSVozP54o,6VNo09sojPBi5mdckQkLbX,King and Cross,brendan.ta,1,Ásgeir,Ásgeir|||||King and Cross
10,brendan.ta,5cOhR878H8hC3UsxYq5Xyv,All My Friends (feat. Tinashe & Chance the Rap...,"[2FwJwEswyIUAljqgjNSHgP, 0NIIxcxNHmOoyBx03SfTC...","[Snakehips, Tinashe, Chance the Rapper]",2016-04-29T05:34:47Z,3pTPXB3vT93AOTSVozP54o,6TaqooOXAEcijL6G1AWS2K,All My Friends (feat. Tinashe & Chance the Rap...,brendan.ta,1,Snakehips___Tinashe___Chance the Rapper,Snakehips___Tinashe___Chance the Rapper|||||Al...
100,brendan.ta,07hs3meNvGu5Fp46pnLQm7,Hungry Ghost,[2N2EFVDEbp2JB8ulEUVIxp],[Violent Soho],2017-02-16T03:09:55Z,3pTPXB3vT93AOTSVozP54o,14NnWDVM8nCssfHAqWZp1T,Covered In Chrome,brendan.ta,1,Violent Soho,Violent Soho|||||Covered In Chrome
1000,brendan.ta,4PKH86wn7Gw4iel2WD564k,Do It Like You,[0aA1GTrIMutjIh4GlPPUVN],[Crooked Colours],2019-02-04T05:22:30Z,3LF6iEL6uCE9rSdLF4m1ga,5h1Gi50T9QxVfzvIF8vciC,Do It Like You,brendan.ta,1,Crooked Colours,Crooked Colours|||||Do It Like You


| Users | Playlists   | Unique Songs   | Total Songs|
|------|------|------|------|
|   918  | 6049| 116157| 515096|

| Avg Playlists/User | Avg Songs/Playlist   |
|-----|------|
|   6.6  | 85.2|

In [21]:
def search_by_artist_and_track(df, artist, track):
    '''
    This function searches the database by artist and track. All tracks that contain substrings with an
    exact match of the artist and track strings are returned. Only the first match is printed.
    '''
    print(df[df.artist_and_track.str.contains(artist, regex=False, case=False) & df.artist_and_track.str.contains(track, regex=False, case=False)].artist_and_track.iloc[0])
    return df[df.artist_and_track.str.contains(artist, regex=False, case=False) & df.artist_and_track.str.contains(track, regex=False, case=False)]



In [22]:
def make_playlist(user_id, playlist_id, df, list_of_searches):
    '''
    Makes a playlist from a list of searches using the first songs returned by each search. Returns a dataframe
    representing the new playlist.
    '''
    new_df = pd.DataFrame()
    for artist,track in list_of_searches:
        curr = search_by_artist_and_track(df, artist, track).iloc[0]
        new_df = new_df.append(pd.DataFrame([[user_id, curr.album_id,curr.album_name,curr.artist_ids,
                   curr.artist_names,'NA',playlist_id,curr.track_id,
                   curr.track_name,user_id,1,curr.artists_join,curr.artist_and_track]], columns=df.columns))
    return new_df

In [23]:
def create_sparse(df):
    '''
    Creates a sparse matrix of the songs and playlists.
    '''
    playlist_id_c = CategoricalDtype(sorted(df.playlist_id.unique()), ordered=True)
    artist_and_track_c = CategoricalDtype(sorted(df.artist_and_track.unique()), ordered=True)

    row = df.playlist_id.astype(playlist_id_c).cat.codes
    col = df.artist_and_track.astype(artist_and_track_c).cat.codes
    sparse_matrix = csr_matrix((df['rating'], (row, col)), \
                               shape=(playlist_id_c.categories.size, artist_and_track_c.categories.size))
    return sparse_matrix, playlist_id_c

In [24]:
def create_sim_dict(playlist_id, df):
    '''
    Creates similarity dictionary from sparse matrix.
    '''
    sparse_matrix, playlist_id_c = create_sparse(df)
    similarities_playlists = cosine_similarity(sparse_matrix)
    playlists = np.array(playlist_id_c.categories)
    playlists_indices = list(playlists)
    sim_zip = zip(playlists_indices, similarities_playlists[playlists_indices.index(playlist_id)])
    sim_dict = {p:sim for p,sim in sim_zip}
    return sim_dict

In [25]:
def create_rec_df(playlist_id, df):
    '''
    Create recommendations with sum of similarities. This should be used over the mean of similarities if you
    want song popularity to influence song recommendations. Higher popularity means the song is more likely to
    be recommended. Returns recommendations, which are the songs listed in order of decreasing similarity sums.
    '''
    df_rec = df[df.playlist_id != playlist_id]
    sim_dict = create_sim_dict(playlist_id, df)
    df_sim = pd.DataFrame(zip(sim_dict.keys(), sim_dict.values()))
    df_sim.columns = ['playlist_id','sim']
    df_rec = df.merge(df_sim,how='left', on='playlist_id')
    grouped = df_rec.groupby(by = 'artist_and_track').sum()
    recommendations = grouped.sort_values(by = 'sim', ascending = False)
    return recommendations

In [26]:
def recommend_for_playlist(user_id, playlist_id, df, list_of_searches):
    '''
    Temporarily adds an input playlist into a dataframe in order to return similarities. Returns recommendations.
    Currently, it's likely that the input playlist songs are included in the recommendations. This is intentional
    in order to set up future work with the spotify api but may be changed in a later version.
    '''
    my_playlist = make_playlist(user_id, playlist_id, df, list_of_searches)
    df = df.append(my_playlist)
    recommendations = create_rec_df(playlist_id, df)
    return recommendations

In [27]:
def simplify(messy_rec, number_tracks):
    '''
    Creates a clean dataframe from recommendations, which is the return from the recommend_for_playlist functions.
    '''
    combined = []
    for row in messy_rec.head(number_tracks).itertuples():
        combined.append(row.Index.split('|||||'))
    clean_df = pd.DataFrame(np.array(combined))
    clean_df.columns = ['artist','track']
    return clean_df

In [28]:
def create_rec_df_mean(playlist_id, df):
    '''
    Create recommendations with mean of similarities. This should be used over the sum of similarities if you
    DO NOT want song popularity to influence song recommendations. This is a good option if you find you're
    getting the same recommendations often. Returns recommendations, which are the songs listed in order of
    decreasing similarity means.
    '''
    df_rec = df[df.playlist_id != playlist_id]
    sim_dict = create_sim_dict(playlist_id, df)
    df_sim = pd.DataFrame(zip(sim_dict.keys(), sim_dict.values()))
    df_sim.columns = ['playlist_id','sim']
    df_rec = df.merge(df_sim,how='left', on='playlist_id')
    grouped = df_rec.groupby(by = 'artist_and_track').mean()
    recommendations = grouped.sort_values(by = 'sim', ascending = False)
    return recommendations

In [29]:
def recommend_for_playlist_mean(user_id, playlist_id, df, list_of_searches):
    '''
    Temporarily adds an input playlist into a dataframe in order to return similarities. Returns recommendations.
    Currently, it's likely that the input playlist songs are included in the recommendations. This is intentional
    in order to set up future work with the spotify api but may be changed in a later version.
    '''
    my_playlist = make_playlist(user_id, playlist_id, df, list_of_searches)
    df = df.append(my_playlist)
    recommendations = create_rec_df_mean(playlist_id, df)
    return recommendations

In [33]:
simplify(recommend_for_playlist('justin', 'justins jams', df, [('croce','i got a name'),
                                                               ('steely dan','do it again'),
                                                              ('spinners','rubberband')]), 30)

Jim Croce|||||I Got A Name
Steely Dan|||||Do It Again
The Spinners|||||The Rubberband Man


,artist,track
0,The Spinners,The Rubberband Man
1,Steely Dan,Do It Again
2,Jim Croce,I Got A Name
3,Electric Light Orchestra,Mr. Blue Sky
4,Looking Glass,Brandy (You're a Fine Girl)
5,Rupert Holmes,Escape (The Pina Colada Song)
6,Creedence Clearwater Revival,Fortunate Son
7,Yusuf / Cat Stevens,Father And Son
8,Boston,More Than a Feeling
9,The Doobie Brothers,Listen to the Music


In [30]:
simplify(recommend_for_playlist_mean('justin', 'justins jams', df, [('croce','i got a name'),
                                                     ('steely dan','do it again')]), 20)

Jim Croce|||||I Got A Name
Steely Dan|||||Do It Again


,artist,track
0,Jim Croce,I Got A Name
1,Steely Dan,Do It Again
2,The Doobie Brothers,Listen to the Music - Single Version; 2006 Rem...
3,Dionne Farris,I Know
4,Titus Andronicus,A More Perfect Union
5,Ambrosia___Tom Trefethen,How Much I Feel - Remastered Version
6,Chicago,Feelin' Stronger Every Day
7,Gary Wright,Dream Weaver
8,Dickey Betts___The Allman Brothers Band,Ramblin' Man
9,Linda Ronstadt,Blue Bayou


In [32]:
simplify(recommend_for_playlist_mean('Dan', 'Dans jams', df, [('led','stairway'),
                                                             ('paul','band on the run')]), 30)

Led Zeppelin|||||Stairway to Heaven - Remaster
Paul McCartney___Wings|||||Band On The Run
The Spinners|||||The Rubberband Man


,artist,track
0,The Spinners,The Rubberband Man
1,Paul McCartney___Wings,Band On The Run
2,The Revivalists,Stand Up
3,Led Zeppelin,Stairway to Heaven - Remaster
4,Randal L Meek,Following Our Dreams
5,John Denver,"Poems, Prayers and Promises - Live at the Univ..."
6,Natural Child,NSA Blues
7,Van Halen,(Oh) Pretty Woman - 2015 Remaster
8,Boston,It's Easy
9,Alabama,Keepin' Up


In [24]:
recommend_for_playlist_mean('Dan', 'Dans jams', df, [('man','killers'),
                                                     ('bette','carnes'),
                                                     ('york groove','ace'),
                                                     ('follow me','muse'),
                                                     ('radio ga','queen'),
                                                     ('i want to break free','queen'),
                                                     ('xanadu','rush')])



The Killers|||||Human
Kim Carnes|||||Bette Davis Eyes
Ace Frehley|||||New York Groove
Muse|||||Follow Me
Queen|||||Radio Ga Ga - Remastered
Queen|||||I Want To Break Free - Remastered
Rush|||||Xanadu


,rating,sim
artist_and_track,,
Queen|||||I Want To Break Free - Remastered,1,0.350852
Muse|||||Follow Me,1,0.223680
Rush|||||Xanadu,1,0.190906
Ace Frehley|||||New York Groove,1,0.122969
T. Rex|||||Mambo Sun - Remastered,1,0.070508
Kim Carnes|||||Bette Davis Eyes,1,0.064655
KISS|||||God Of Thunder - Demo Version,1,0.064628
KISS|||||Psycho Circus - Without Intro,1,0.064628
KISS|||||Nothing Can Keep Me From You,1,0.064628


In [13]:
test_recs = recommend_for_playlist('JSON', 'jasons jams', df, [('future Islands', 'Dream'), ('Beach Fossils','Down')])



Future Islands|||||A Dream of You and Me
Beach Fossils|||||Down the Line


In [205]:
search_by_artist_and_track(df, 'Overnight','Maggie')

Maggie Rogers|||||Overnight


,added_by_id,album_id,album_name,artist_ids,artist_names,date_added,playlist_id,track_id,track_name,user_id,rating,artists_join,artist_and_track
362930,if_u_like_playlists,5AHWNPo3gllDmixgAoFru4,Heard It In A Past Life,[4NZvixzsSefsNiIqXn0NDe],[Maggie Rogers],2019-02-03T16:22:02Z,4smNayOKlCtwUAIPOfsKOK,4L9l5x0WuzNtyEBeqzvgPE,Overnight,if_u_like_playlists,1,Maggie Rogers,Maggie Rogers|||||Overnight
500144,hipsterthemister,5AHWNPo3gllDmixgAoFru4,Heard It In A Past Life,[4NZvixzsSefsNiIqXn0NDe],[Maggie Rogers],2019-01-19T16:30:21Z,2TwfFafDftf4WjpZTp00Sr,4L9l5x0WuzNtyEBeqzvgPE,Overnight,hipsterthemister,1,Maggie Rogers,Maggie Rogers|||||Overnight
500242,hipsterthemister,5AHWNPo3gllDmixgAoFru4,Heard It In A Past Life,[4NZvixzsSefsNiIqXn0NDe],[Maggie Rogers],2019-01-18T16:09:16Z,0dnGaIHv1Y85bQla9LvxXE,4L9l5x0WuzNtyEBeqzvgPE,Overnight,hipsterthemister,1,Maggie Rogers,Maggie Rogers|||||Overnight
719422,ram0nn7,5AHWNPo3gllDmixgAoFru4,Heard It In A Past Life,[4NZvixzsSefsNiIqXn0NDe],[Maggie Rogers],2019-01-24T18:12:58Z,5QaYHdUWlOeRb4EVl2ipWv,4L9l5x0WuzNtyEBeqzvgPE,Overnight,stoneygatesound,1,Maggie Rogers,Maggie Rogers|||||Overnight


In [185]:
simplify(jasons_recs, 20)

,artist,track
0,Beach Fossils,Down the Line
1,Future Islands,A Dream of You and Me
2,Peach Pit,Peach Pit
3,Tame Impala,The Less I Know The Better
4,Good Morning,Warned You
5,boy pablo,Everytime
6,Beach Fossils,What a Pleasure
7,Tame Impala,Feels Like We Only Go Backwards
8,Mac DeMarco,My Kind of Woman
9,The Drums,Days


In [177]:
jasons_recs.head(20)

,rating,sim
artist_and_track,,
Beach Fossils|||||Down the Line,20,2.207815
Future Islands|||||A Dream of You and Me,4,1.205020
Peach Pit|||||Peach Pit,20,0.619935
Tame Impala|||||The Less I Know The Better,99,0.452416
Good Morning|||||Warned You,25,0.404688
boy pablo|||||Everytime,25,0.375735
Beach Fossils|||||What a Pleasure,13,0.369210
Tame Impala|||||Feels Like We Only Go Backwards,69,0.357343
Mac DeMarco|||||My Kind of Woman,15,0.355247


In [152]:
coles_recs.head(20)

,rating,sim,weighted?
artist_and_track,,,
Masego___FKJ|||||Tadow,54,0.479657,0.008883
Childish Gambino|||||Redbone,127,0.443397,0.003491
Nick Hakim|||||Cuffed,19,0.346619,0.018243
Steve Lacy|||||Some,23,0.305193,0.013269
Matt Martians___Syd___Steve Lacy|||||Dent Jusay,10,0.300350,0.030035
Frank Ocean|||||Nikes,38,0.299221,0.007874
Masego|||||Navajo,27,0.283595,0.010504
BJ The Chicago Kid|||||Turnin' Me Up,22,0.272489,0.012386
Sampa the Great|||||Blue Boss,11,0.263032,0.023912


In [101]:
search_by_artist_and_track(df, 'Thundercat', 'Them Changes').iloc[0]

added_by_id                           patreeeek
album_id                 4anxEzh8xEKSPXKHGWiFue
album_name                                Drunk
artist_ids             [4frXpPxQQZwbCu3eTGnZEw]
artist_names                       [Thundercat]
date_added                 2018-06-16T20:27:17Z
playlist_id              0OLCESUYzhIboDmPv9XRGf
track_id                 5dcZUsgcMW57ju0A379O7c
track_name                         Them Changes
user_id                               patreeeek
rating                                        1
artists_join                         Thundercat
artist_and_track    Thundercat|||||Them Changes
Name: 123085, dtype: object

In [108]:
cole_df = make_playlist('test_5', 'doesnt matter', df, [('Thundercat', 'Them Changes'), ('Girlfriend Is Better','Talking Heads')])

In [109]:
cole_df

,added_by_id,album_id,album_name,artist_ids,artist_names,date_added,playlist_id,track_id,track_name,user_id,rating,artists_join,artist_and_track
0,test_5,4anxEzh8xEKSPXKHGWiFue,Drunk,[4frXpPxQQZwbCu3eTGnZEw],[Thundercat],NA,doesnt matter,5dcZUsgcMW57ju0A379O7c,Them Changes,test_5,1,Thundercat,Thundercat|||||Them Changes
0,test_5,4sLCQxMRfn3gAHrBNZtbTH,Speaking In Tongues (Deluxe Version),[2x9SpqnPi8rlE9pjHBwmSC],[Talking Heads],NA,doesnt matter,6VyR9z26oKw6BvlwSjqvoY,Girlfriend Is Better - 2005 Remastered Version,test_5,1,Talking Heads,Talking Heads|||||Girlfriend Is Better - 2005 ...


In [95]:
search_by_artist_and_track(df, 'Girlfriend Is Better', 'Talking Heads')

,added_by_id,album_id,album_name,artist_ids,artist_names,date_added,playlist_id,track_id,track_name,user_id,rating,artists_join,artist_and_track
142614,patreeeek,4sLCQxMRfn3gAHrBNZtbTH,Speaking In Tongues (Deluxe Version),[2x9SpqnPi8rlE9pjHBwmSC],[Talking Heads],2019-01-20T01:58:35Z,2HJy3TNqrRXr6DMvXFGCxN,6VyR9z26oKw6BvlwSjqvoY,Girlfriend Is Better - 2005 Remastered Version,patreeeek,1,Talking Heads,Talking Heads|||||Girlfriend Is Better - 2005 ...
161383,patreeeek,78MM8HrabEGPLVWaJkM2t1,Speaking In Tongues,[2x9SpqnPi8rlE9pjHBwmSC],[Talking Heads],2013-03-09T15:41:44Z,0KRN0GjJaYn4uc2daHTWBx,37kkHqpkzTEGOJYHRb6JZG,Girlfriend Is Better,patreeeek,1,Talking Heads,Talking Heads|||||Girlfriend Is Better
163370,patreeeek,4sLCQxMRfn3gAHrBNZtbTH,Speaking In Tongues (Deluxe Version),[2x9SpqnPi8rlE9pjHBwmSC],[Talking Heads],2019-04-02T17:59:20Z,5CzjSMJf8z9pp2ek0xCGmM,6VyR9z26oKw6BvlwSjqvoY,Girlfriend Is Better - 2005 Remastered Version,patreeeek,1,Talking Heads,Talking Heads|||||Girlfriend Is Better - 2005 ...
423606,cannondale12,17K4P45WAJU1GeCqAIC8OE,Speaking In Tongues (Deluxe Version),[2x9SpqnPi8rlE9pjHBwmSC],[Talking Heads],2014-03-14T00:37:16Z,428zCMVCQGMmbCCYawqsBV,2UYY46dgNshqDfDF20RBYM,Girlfriend Is Better - 2005 Remastered Version,cannondale12,1,Talking Heads,Talking Heads|||||Girlfriend Is Better - 2005 ...
543316,21n2ku53disfrt53dgeg3vbki,78MM8HrabEGPLVWaJkM2t1,Speaking In Tongues,[2x9SpqnPi8rlE9pjHBwmSC],[Talking Heads],2017-02-17T22:43:15Z,58fnmHO3IapTcllIDF4P4p,37kkHqpkzTEGOJYHRb6JZG,Girlfriend Is Better,21n2ku53disfrt53dgeg3vbki,1,Talking Heads,Talking Heads|||||Girlfriend Is Better


In [92]:
search_by_artist_and_track(df, 'Thundercat', 'Them Changes')

,added_by_id,album_id,album_name,artist_ids,artist_names,date_added,playlist_id,track_id,track_name,user_id,rating,artists_join,artist_and_track
123085,patreeeek,4anxEzh8xEKSPXKHGWiFue,Drunk,[4frXpPxQQZwbCu3eTGnZEw],[Thundercat],2018-06-16T20:27:17Z,0OLCESUYzhIboDmPv9XRGf,5dcZUsgcMW57ju0A379O7c,Them Changes,patreeeek,1,Thundercat,Thundercat|||||Them Changes
150551,patreeeek,4anxEzh8xEKSPXKHGWiFue,Drunk,[4frXpPxQQZwbCu3eTGnZEw],[Thundercat],2019-02-02T02:36:34Z,0OMJq2zGLIxy18hZVUxO5y,5dcZUsgcMW57ju0A379O7c,Them Changes,patreeeek,1,Thundercat,Thundercat|||||Them Changes
216399,1126708296,1atZ1TiIPZZec0oiyY1mBl,The Beyond / Where the Giants Roam,"[4frXpPxQQZwbCu3eTGnZEw, 29XOeO6KIWxGthejQqn79...","[Thundercat, Flying Lotus, Kamasi Washington]",2018-09-27T22:26:16Z,3Wyjv35Hgqeq71kmLL85wo,4u8031pTkTODbRaDCbx8Yv,Them Changes,1126708296,1,Thundercat___Flying Lotus___Kamasi Washington,Thundercat___Flying Lotus___Kamasi Washington|...
326714,1214486915,4anxEzh8xEKSPXKHGWiFue,Drunk,[4frXpPxQQZwbCu3eTGnZEw],[Thundercat],2018-11-13T05:29:07Z,1k73xi3WCoRSHSm7bEq1Vf,5dcZUsgcMW57ju0A379O7c,Them Changes,1214486915,1,Thundercat,Thundercat|||||Them Changes
345045,hilltopchill,7vgnnPWflv8g0vQYOberLn,The Beyond / Where the Giants Roam,"[4frXpPxQQZwbCu3eTGnZEw, 29XOeO6KIWxGthejQqn79...","[Thundercat, Flying Lotus, Kamasi Washington]",2019-02-05T10:01:55Z,1RaurPqzifj6NIgnP1Zlt0,7hmiDaBxoyLW094H8vup2S,Them Changes (feat. Flying Lotus & Kamasi Wash...,hilltopchill,1,Thundercat___Flying Lotus___Kamasi Washington,Thundercat___Flying Lotus___Kamasi Washington|...
346332,odrey918,7vHBQDqwzB7uDvoE5bncMM,Drunk,[4frXpPxQQZwbCu3eTGnZEw],[Thundercat],2019-02-22T20:32:33Z,2PKHcWNVsjljkpb9XcVodl,7CH99b2i1TXS5P8UUyWtnM,Them Changes,odrey918,1,Thundercat,Thundercat|||||Them Changes
393411,tmacgordon5,4anxEzh8xEKSPXKHGWiFue,Drunk,[4frXpPxQQZwbCu3eTGnZEw],[Thundercat],2018-09-26T03:08:25Z,1t6jUps8IXqPKC7E2GBxIM,5dcZUsgcMW57ju0A379O7c,Them Changes,tmacgordon5,1,Thundercat,Thundercat|||||Them Changes
411012,devotron,4anxEzh8xEKSPXKHGWiFue,Drunk,[4frXpPxQQZwbCu3eTGnZEw],[Thundercat],2018-09-24T15:57:12Z,03DQE6raYvdocaTRjby1Wh,5dcZUsgcMW57ju0A379O7c,Them Changes,lg7gkjmevh6brhby94ofq7a4m,1,Thundercat,Thundercat|||||Them Changes
416680,1259677891,4anxEzh8xEKSPXKHGWiFue,Drunk,[4frXpPxQQZwbCu3eTGnZEw],[Thundercat],2017-11-08T11:37:23Z,31gz6eKGfwE01EcXTrCHUs,5dcZUsgcMW57ju0A379O7c,Them Changes,1259677891,1,Thundercat,Thundercat|||||Them Changes
543763,21n2ku53disfrt53dgeg3vbki,1atZ1TiIPZZec0oiyY1mBl,The Beyond / Where the Giants Roam,"[4frXpPxQQZwbCu3eTGnZEw, 29XOeO6KIWxGthejQqn79...","[Thundercat, Flying Lotus, Kamasi Washington]",2017-02-17T22:44:50Z,1nCIatfvCg9jh17THwweXe,4u8031pTkTODbRaDCbx8Yv,Them Changes,21n2ku53disfrt53dgeg3vbki,1,Thundercat___Flying Lotus___Kamasi Washington,Thundercat___Flying Lotus___Kamasi Washington|...


In [69]:
df2 = pd.DataFrame([['test_added2', '5CZR6ljD0x9fTiS4mh9wMp','Only By The Night','[2qk9voo8llSGYcZ6xrBzKx]','[Kings of Leon]','2017-08-23T18:14:04Z','test_playlist2','0ntQJM78wzOLVeCUAW7Y45','Sex on Fire','test_user2',1,'Kings of Leon','Kings of Leon|||||Sex on Fire']], columns=df.columns)
df2

,added_by_id,album_id,album_name,artist_ids,artist_names,date_added,playlist_id,track_id,track_name,user_id,rating,artists_join,artist_and_track
0,test_added2,5CZR6ljD0x9fTiS4mh9wMp,Only By The Night,[2qk9voo8llSGYcZ6xrBzKx],[Kings of Leon],2017-08-23T18:14:04Z,test_playlist2,0ntQJM78wzOLVeCUAW7Y45,Sex on Fire,test_user2,1,Kings of Leon,Kings of Leon|||||Sex on Fire


In [70]:
df2.dtypes

added_by_id         object
album_id            object
album_name          object
artist_ids          object
artist_names        object
date_added          object
playlist_id         object
track_id            object
track_name          object
user_id             object
rating               int64
artists_join        object
artist_and_track    object
dtype: object

In [71]:
df = df.append(df2)

In [73]:
sparse_matrix

<6051x116157 sparse matrix of type '<class 'numpy.int64'>'
	with 489351 stored elements in Compressed Sparse Row format>

In [74]:
similarities_playlists = cosine_similarity(sparse_matrix)

In [75]:
playlists = np.array(playlist_id_c.categories)

In [76]:
playlists_indices = list(playlists)

In [77]:
playlists_indices

['002MBUoZVJntSIsgW6EpFA',
 '0061U3t8hRFhIBw42to0Yw',
 '008wDZ2VZxGTRYLYS82mya',
 '00AP7VyekxNKmWiMj9xisM',
 '00CsN1rCRTN8tKcPKcHCjQ',
 '00CzKdtJgJelMbJJvs2rJN',
 '00LP85DvUnR9CHeqAFv7VL',
 '00SrfsM0Wh4ZfZSrA9OQS3',
 '00UWUyGDpyN7bhsoUZVqdo',
 '00Vv9ktcShVGl695Nqtqjn',
 '00WPgq2AQvZ6vaXnaUddy0',
 '00b9NrNWGW7kB3mlk6lTUd',
 '00nBF6hbRoRLUKp2WvRrtF',
 '00rjDqOCrMtgV7wcxScgHM',
 '00wHFIu00pH4CJWPV4vJwA',
 '01AAC8Hn9BqioBD4AITbYN',
 '01AyypY3Vg6BZ7Fvbpi8AO',
 '01Bj96ebCNSskOmkh8taPP',
 '01GwqHIWoJmhLKfRhu5uvq',
 '01Ivk66sjv7fxG28mSEC3B',
 '01JIbtNjFwWjuddNL0Tq71',
 '01U7nKpzTRK1Hp2V03IMWu',
 '01UbuTuMY0TvMz42KXoy3y',
 '01VpWWi8xzzS6nJ64J5hQA',
 '01WaTuwNZnqwbBkRk2UeG9',
 '01aCMPN9vISPWsMqQtol8S',
 '01gjluhMJeFT0WcvR1937w',
 '01gyv6qrcXgycqwnbhMHO2',
 '01oOxpTnnokNxu5AitOL41',
 '01qHdqpm6S7zv5N3vChwye',
 '01rNo73R3xjRpa3G0VbKVH',
 '01tvqPxen74Nz7TRz7kpSd',
 '01wyjmhDW4Fru5cb0TQ7Dp',
 '023J3zoqmmqR0cUp7hyAcw',
 '02NUIaAZADXg8qYnwCezev',
 '02Tt4tSbfwN8EYTMc3ZzgM',
 '02Yypb6ck9z09yWm41yU5l',
 

In [78]:
playlists_indices.index('test_playlist2')

6050

In [79]:
similarities_playlists[6050]

array([0., 0., 0., ..., 0., 0., 1.])

In [17]:
# np.save('playlist_similarities',similarities_playlists)

In [20]:
test_dict = create_sim_dict('test_playlist')

In [52]:
recs = create_rec_df('test_playlist', df)

In [55]:
recs

,rating,sim
artist_and_track,,
Kendrick Lamar___Jay Rock|||||Money Trees,43,4.764937
"Kendrick Lamar|||||Bitch, Don’t Kill My Vibe",54,1.814687
Kendrick Lamar|||||HUMBLE.,95,1.379705
Kendrick Lamar|||||DNA.,76,1.341237
Kendrick Lamar|||||ELEMENT.,39,1.219696
Kendrick Lamar|||||Backseat Freestyle,37,1.202267
Kendrick Lamar___Zacari|||||LOVE. FEAT. ZACARI.,74,1.158540
Kendrick Lamar|||||Swimming Pools (Drank) - Extended Version,32,1.139217
Kendrick Lamar___MC Eiht|||||m.A.A.d city,52,1.103397


In [32]:
df.head()

,added_by_id,album_id,album_name,artist_ids,artist_names,date_added,playlist_id,track_id,track_name,user_id,rating,artists_join,artist_and_track
0,brendan.ta,4uIDigk79DeZEYV6Z5Yf4s,What Went Down,[6FQqZYVfTNQ1pCqfkwVFEa],[Foals],2016-03-26T00:48:16Z,3pTPXB3vT93AOTSVozP54o,53L6A3I9vf7rgEZnMzx54E,Mountain At My Gates,brendan.ta,1,Foals,Foals|||||Mountain At My Gates
1,brendan.ta,4sFhah3DYcJlYeT47q3rhM,In The Silence (Deluxe Version),[7xUZ4069zcyBM4Bn10NQ1c],[Ásgeir],2016-03-26T00:49:53Z,3pTPXB3vT93AOTSVozP54o,6VNo09sojPBi5mdckQkLbX,King and Cross,brendan.ta,1,Ásgeir,Ásgeir|||||King and Cross
10,brendan.ta,5cOhR878H8hC3UsxYq5Xyv,All My Friends (feat. Tinashe & Chance the Rap...,"[2FwJwEswyIUAljqgjNSHgP, 0NIIxcxNHmOoyBx03SfTC...","[Snakehips, Tinashe, Chance the Rapper]",2016-04-29T05:34:47Z,3pTPXB3vT93AOTSVozP54o,6TaqooOXAEcijL6G1AWS2K,All My Friends (feat. Tinashe & Chance the Rap...,brendan.ta,1,Snakehips___Tinashe___Chance the Rapper,Snakehips___Tinashe___Chance the Rapper|||||Al...
100,brendan.ta,07hs3meNvGu5Fp46pnLQm7,Hungry Ghost,[2N2EFVDEbp2JB8ulEUVIxp],[Violent Soho],2017-02-16T03:09:55Z,3pTPXB3vT93AOTSVozP54o,14NnWDVM8nCssfHAqWZp1T,Covered In Chrome,brendan.ta,1,Violent Soho,Violent Soho|||||Covered In Chrome
1000,brendan.ta,4PKH86wn7Gw4iel2WD564k,Do It Like You,[0aA1GTrIMutjIh4GlPPUVN],[Crooked Colours],2019-02-04T05:22:30Z,3LF6iEL6uCE9rSdLF4m1ga,5h1Gi50T9QxVfzvIF8vciC,Do It Like You,brendan.ta,1,Crooked Colours,Crooked Colours|||||Do It Like You


In [86]:
recs = create_rec_df('test_playlist2', df)

In [87]:
recs

,rating,sim
artist_and_track,,
Kings of Leon|||||Sex on Fire,57,5.506253
The Killers|||||Mr. Brightside,107,1.757159
Kings of Leon|||||Use Somebody,49,1.657102
The White Stripes|||||Seven Nation Army,88,1.534124
Franz Ferdinand|||||Take Me Out,90,1.457202
The Killers|||||Somebody Told Me,53,1.209054
The Strokes|||||Last Nite,65,1.195187
Arctic Monkeys|||||Do I Wanna Know?,89,1.070059
Red Hot Chili Peppers|||||Californication,82,1.047806
